Francisco Javier Piqueras Martínez

# Practica Programación en Entorno de datos 2020-2021

En esta práctica se plantea un caso práctico que se tendrá que resolver con los contenidos vistos en el curso. Se puede buscar y utilizar funciones adicionales que se crea necesario, justificando su elección y explicando su uso.

La práctica se realizará sobre un notebook, que debe contener:
- El enunciado de cada apartado que se resuelva
- Justificación de todas las decisiones tomadas
- Código utilizado, mostrando el resultado de su ejecución, para responder a todas las preguntas que se realizan. **Importante:** El código debe funcionar en cualquier máquina sin necesidad de instalar nada. Para ello es importante usar rutas relativas y no absolutas.
- Cualesquiera comentario que se considere necesario.

## Enunciado

El juego de los euromillones es un juego de azar que consiste en elegir en cada apuesta 5 números de entre 50 (del número 1 al 50, ambos inclusive) sin repeticiones, y dos números de estrellas de entre 12 (del 1 al 12, ambos inclusive) sin repeticiones. En total, para cada apuesta tenemos 7 números (por ejemplo, la combinación ganadora del pasado 16 de octubre de 2020 fue: 15, 33, 38, 40, 50 – 3, 6).

Los premios están divididos en 13 categorías en función de la cantidad de números y estrellas que se acierten de acuerdo a la tabla siguiente:

|Categoría|Números acertados|Estrellas acertadas|
|---|---|---|
|1|5|2|
|2|5|1|
|3|5|0|
|4|4|2|
|5|4|1|
|6|3|2|
|7|4|0|
|8|2|2|
|9|3|1|
|10|3|0|
|11|1|2|
|12|2|1|
|13|2|0|

Junto con el enunciado se distribuye un fichero (resultadosEuromillones.csv) donde está guardado el histórico de las apuestas de euromillones ganadoras junto con su correspondiente fecha

## Comentarios del estudiante sobre el enunciado

> Nota: A pesar de indicar que el codigo debe funcionar en cualquier máquina sin necesidad de instalar nada, se asume que se tiene instalado **python, jupyter, pandas y numpy.**

En primer lugar realizamos los imports necesarios para trabajar en la práctica:

In [1]:
import numpy as np
import pandas as pd
import os.path
import random

## Realización de los ejercicios

### Ejercicio 1
**Explora el fichero y decide cómo tienes que realizar la importación para poder guardar la información. ¿En qué estructura lo vas a guardar y qué información concreta contiene? Escribe el código asociado y trata de optimizar el espacio utilizado.**

Leemos los datos con la función `read_csv` de pandas:

In [2]:
datos = pd.read_csv('historicoEuromillones.csv')

Y los observamos:

In [3]:
datos.head()

,FECHA,COMB. GANADORA,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,ESTRELLAS,Unnamed: 8
0,13/10/2020,5,14,38,41,46,NaN,1,10
1,9/10/2020,11,15,35,41,50,NaN,5,8
2,6/10/2020,4,21,36,41,47,NaN,9,11
3,2/10/2020,6,12,15,40,45,NaN,3,9
4,29/09/2020,4,14,16,41,44,NaN,11,12


Como podemos observar:

- En la primera columna tenemos la fecha de sorteo: **`FECHA`**
- En la primera columna tenemos la fecha de sorteo: **`FECHA`**
- Desde la segunda columna (**`COMB. GANADORA`**) hasta la sexta columna (**`Unnamed: 5`**) tenemos el valor de cada uno de los números acertados.
- En la séptima columna (**`Unnamed: 6`**), todos los valores son NaN.
- Desde la octava columna (**`ESTRELLAS`**) hasta la novena (**`Unnamed: 8`**) tenemos el valor de cada uno de las estrellas acertadas.


Antes de realizar cualqueir modificación en los datos, vamos a observar la memoria que consume el DataFrame cargado pues vamos a tratar de optimizarlo al máximo: 

In [4]:
datos.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1363 entries, 0 to 1362
Data columns (total 9 columns):
FECHA             1363 non-null object
COMB. GANADORA    1363 non-null int64
Unnamed: 2        1363 non-null int64
Unnamed: 3        1363 non-null int64
Unnamed: 4        1363 non-null int64
Unnamed: 5        1363 non-null int64
Unnamed: 6        0 non-null float64
ESTRELLAS         1363 non-null int64
Unnamed: 8        1363 non-null int64
dtypes: float64(1), int64(7), object(1)
memory usage: 174.1 KB


A continuación, como se ha podido comprobar, todos los valores de la columna `Unnamed: 6` son null, no obstante, si quisieramos comprobarlo por nosotros mismos:

In [5]:
datos['Unnamed: 6'].isnull().all()

True

Realizamos un filtro sobre el `Series` devuelto por la columna 6 que devolverá el valor `True` a aquellas filas cuyo valor sea `NaN` y `False` al resto. Seguidamente, sometemos el resultado a la función all() que devuelve `True` si todos los valores son `True` y `False` en caso contrario.

Como ha quedado demostrado, todos los valores de la columna en cuestión eran `null`, por lo que podemos eliminarla.

Antes de empezar a trabajar, realizamos una copia de los datos para poder comparar más tarde (lo hacemos con el método `copy()` para crear una copia del objeto, sino, cualquier modificación hecha en `datos` se verá reflejada también en `datos_original` ya que ambas variables haría referencia al mismo objeto):

In [6]:
datos_original = datos.copy()

Para poder tener la información organizada de una forma más intuitiva, vamos a realizar un renombrado de las columnas:

In [7]:
datos.rename(columns={'COMB. GANADORA': 'COMB1',
            'Unnamed: 2': 'COMB2',
            'Unnamed: 3': 'COMB3',
            'Unnamed: 4': 'COMB4',
            'Unnamed: 5': 'COMB5',
            'ESTRELLAS': 'EST1',
            'Unnamed: 8': 'EST2'}, inplace=True)

Y eliminamos la columna cuyos valores son todos nulos:

In [8]:
datos.drop(columns=['Unnamed: 6'], inplace=True)

Ahora, vamos a crear la estructura de nuestro nuevo DataFrame, este va a tener 5 columnas:
- `DAY`: El día del sorte del mes
- `MONTH`: El mes del sorteo
- `YEAR`: El año del sorteo
- `TYPE`: El tipo de número (`Star` o `Number`)
- `NUMBER`: El numero que ha salido

Esta nueva distribución se hace con el fin de ahorrar espacio y de mejorar el procesamiento que se va a necesitar después. **Tarda un poco en procesar. No obstante, en cuanto hemos realizado la conversión una vez, exportamos el DataFrame para cargarlo en futuras ocasiones.**

> **Nota**: *En cuanto al procesamiento, lo ideal sería trabajar directamente con `set`. No obstante, aumentaría mucho el espacio de nuestro DataFrame:*

In [9]:
def parseDataFrame(df):
    datos_ = pd.DataFrame(columns=['FECHA', 'TYPE', 'NUMBER'])
    for index, row in df.iterrows():
        datos_ = datos_.append({'FECHA': row['FECHA'], 'TYPE': 'Number', 'NUMBER': row['COMB1']}, ignore_index=True)
        datos_ = datos_.append({'FECHA': row['FECHA'], 'TYPE': 'Number', 'NUMBER': row['COMB2']}, ignore_index=True)
        datos_ = datos_.append({'FECHA': row['FECHA'], 'TYPE': 'Number', 'NUMBER': row['COMB3']}, ignore_index=True)
        datos_ = datos_.append({'FECHA': row['FECHA'], 'TYPE': 'Number', 'NUMBER': row['COMB4']}, ignore_index=True)
        datos_ = datos_.append({'FECHA': row['FECHA'], 'TYPE': 'Number', 'NUMBER': row['COMB5']}, ignore_index=True)
        datos_ = datos_.append({'FECHA': row['FECHA'], 'TYPE': 'Star', 'NUMBER': row['EST1']}, ignore_index=True)
        datos_ = datos_.append({'FECHA': row['FECHA'], 'TYPE': 'Star', 'NUMBER': row['EST2']}, ignore_index=True) 
    datos = datos_
    datos['DAY'] = np.array(datos['FECHA'].str.split('/').tolist())[:, 0]
    datos['MONTH'] = np.array(datos['FECHA'].str.split('/').tolist())[:, 1]
    datos['YEAR'] = np.array(datos['FECHA'].str.split('/').tolist())[:, 2]
    # Reordenamos las columnas unicamente por mejorar la visualización de datos y eliminamos la columna fecha
    datos = datos[['DAY', 'MONTH', 'YEAR', 'TYPE', 'NUMBER']]
    datos.to_csv('dataframe_parsed.csv', index=False)
    return datos

Comprobamos si existe el archivo exportado anteriormente y lo cargamos. En caso contrario, procesamos el dataFrame original para formatearlo como nos interesa:

> **En este ejercicio se realiza un export ya que modificar el dataframe original y dejarlo con la estructura de interés toma unos segundos. Este archivo se ha incluído con la práctica (dataframe_parsed.csv). Automáticamente lo cargará en lugar de hacer el procesaro. Si se desea se puede eliminar y se volverá a crear al ejecutarlo.**

In [10]:
if os.path.isfile('dataframe_parsed.csv'):
    datos = pd.read_csv('dataframe_parsed.csv')
else:
    datos = parseDataFrame(datos)

In [11]:
datos.head()

,DAY,MONTH,YEAR,TYPE,NUMBER
0,13,10,2020,Number,5
1,13,10,2020,Number,14
2,13,10,2020,Number,38
3,13,10,2020,Number,41
4,13,10,2020,Number,46


Como se comenta en el enunciado y como se ve a continuación gracias al método `describe`: 
- El máximo valor que pueden tener las columnas `NUMBER` es de 50. Además, este no puede ser negativo. 
- En cuanto a `TYPE` este solamente puede tener el valor `Number` o `Star`

Por lo tanto, convertimos el tipo de datos a `int8`.

En cuanto a las fechas, vamos tambien a convertir el dia y mes en `uint8` y el año en `uint16`. Esto, además de optimizar el dataframe, nos ayudará mucho a la hora de trabajar con las operaciones que realizaremos posteriormente.

In [12]:
datos['NUMBER'] = datos['NUMBER'].astype('uint8')
datos['TYPE'] = datos['TYPE'].astype('category')
datos['DAY'] = datos['DAY'].astype('uint8')
datos['MONTH'] = datos['MONTH'].astype('uint8')
datos['YEAR'] = datos['YEAR'].astype('uint16')

In [13]:
datos.describe()

,DAY,MONTH,YEAR,NUMBER
count,9541.000000,9541.000000,9541.000000,9541.000000
mean,15.733676,6.520910,2013.256053,19.783985
std,8.794273,3.405986,4.536683,15.113934
min,1.000000,1.000000,2004.000000,1.000000
25%,8.000000,4.000000,2010.000000,7.000000
50%,16.000000,7.000000,2014.000000,15.000000
75%,23.000000,9.000000,2017.000000,33.000000
max,31.000000,12.000000,2020.000000,50.000000


In [14]:
datos.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9541 entries, 0 to 9540
Data columns (total 5 columns):
DAY       9541 non-null uint8
MONTH     9541 non-null uint8
YEAR      9541 non-null uint16
TYPE      9541 non-null category
NUMBER    9541 non-null uint8
dtypes: category(1), uint16(1), uint8(3)
memory usage: 56.2 KB


Y finalmente comparamos la memoria actual con la anterior. Como vemos, hay una gran diferencia, hemos reducido el espacio que ocupa nuestro dataframe en 3.1 veces más pequeño (174.1 / 56.2)

Mostramos 5 filas (las del final) para ver como quedaría nuestro DataFrame:

In [15]:
datos.tail()

,DAY,MONTH,YEAR,TYPE,NUMBER
9536,13,2,2004,Number,32
9537,13,2,2004,Number,36
9538,13,2,2004,Number,41
9539,13,2,2004,Star,7
9540,13,2,2004,Star,9


### Ejercicio 2

**Queremos obtener algunas estadísticas relativas al histórico. Concretamente, ¿cuál es la moda, mediana y media de los números? ¿y de las estrellas? ¿puedes obtenerlo para cada año? ¿y para cada mes? Nota: cuando decimos cada mes, nos referimos a todos los meses de, por ejemplo, enero en conjunto. No al mes de enero de 2018 por un lado, al mes de enero de 2017 por otro lado, etc.**

**Escribe el código asociado para realizar todos estos cálculos.**

En primer lugar, la mediana y media para cada uno de los datos se puede obtener a través del método `describe()`. No obstante, se pueden también calcular a mano:

In [16]:
datos.describe()

,DAY,MONTH,YEAR,NUMBER
count,9541.000000,9541.000000,9541.000000,9541.000000
mean,15.733676,6.520910,2013.256053,19.783985
std,8.794273,3.405986,4.536683,15.113934
min,1.000000,1.000000,2004.000000,1.000000
25%,8.000000,4.000000,2010.000000,7.000000
50%,16.000000,7.000000,2014.000000,15.000000
75%,23.000000,9.000000,2017.000000,33.000000
max,31.000000,12.000000,2020.000000,50.000000


Separamos los datos en dos DataFrames según Estrellas o Números para facilitar las operaciones:

In [17]:
datos_numeros = datos[datos['TYPE']=='Number']
datos_estrellas = datos[datos['TYPE']=='Star']

La media, mediana y moda de los números es:

In [18]:
# Media
datos_numeros[['NUMBER']].mean()

NUMBER    25.344828
dtype: float64

In [19]:
# Mediana
datos_numeros[['NUMBER']].median()

NUMBER    25.0
dtype: float64

In [20]:
# Moda
datos_numeros[['NUMBER']].mode()

,NUMBER
0,23


Lo mismo para las estrellas:

In [21]:
# Media
datos_estrellas[['NUMBER']].mean()

NUMBER    5.881878
dtype: float64

In [22]:
# Mediana
datos_estrellas[['NUMBER']].median()

NUMBER    6.0
dtype: float64

In [23]:
# Moda
datos_estrellas[['NUMBER']].mode()

,NUMBER
0,2


Para obtenerlo para cada año va a ser bastante sencillo gracias a como hemos estructurado y dividido las fechas. Lo mismo para sacar los datos por meses:

Un ejemplo de como obtenerlo sería de la siguiente forma:

``
datos.groupby(['YEAR', 'TYPE']).mean()[['NUMBER']]
``

Con esto, podemos definir una función que nos devuelva lo que necesitamos:

In [24]:
def statistics_by(by, operation):
    if by in ['YEAR', 'MONTH']:
        if operation == 'mean':
            return datos.groupby([by, 'TYPE']).mean()[['NUMBER']]
        elif operation == 'median':
            return datos.groupby([by, 'TYPE']).mean()[['NUMBER']]
        elif operation == 'mode':
            return datos.groupby([by, 'TYPE'])[['NUMBER']].apply(lambda x: x.mode())
    else:
        return 0

In [25]:
#statistics_by('MONTH', 'median')
#statistics_by('YEAR', 'median')
#statistics_by('MONTH', 'mean')
#statistics_by('YEAR', 'mean')
#statistics_by('MONTH', 'mode')
statistics_by('YEAR', 'mode')

NUMBER
YEAR TYPE            
2004 Number 0       1
     Star   0       5
2005 Number 0      11
     Star   0       1
2006 Number 0      50
     Star   0       3
2007 Number 0      41
     Star   0       6
            1       8
2008 Number 0      50
     Star   0       4
            1       5
            2       7
2009 Number 0      20
     Star   0       3
            1       5
2010 Number 0      38
            1      46
     Star   0       7
2011 Number 0      12
     Star   0       2
2012 Number 0      10
     Star   0       2
2013 Number 0      13
            1      28
            2      42
     Star   0       2
2014 Number 0      13
     Star   0       1
            1      10
2015 Number 0      30
     Star   0       8
2016 Number 0      37
     Star   0       2
2017 Number 0      17
            1      30
     Star   0       3
2018 Number 0      15
     Star   0      12
2019 Number 0       1
     Star   0       2
2020 Number 0      11
     Star   0       6

Como se puede comprobar, el cálculo de la moda es distinto, ya que `DataFrameGroupBy` no tiene el método `mode`, por lo que hay que hacerlo de forma diferente. En lugar de efectuar la operación sobre el objeto `DataFrameGroupBy`, lo que se ha hecho ha sido ejecutar la función `apply`, en la que como argumento le hemos pasado una función anónima que dentro llama a la función `mode`.

> *Nótese que puede haber más de una moda*

### Ejercicio 3
**¿Cuáles han sido los 5 números más repetidos? ¿y las 2 estrellas? Responde a las mismas preguntas para cada año y cada mes (entendiendo mes igual que en el ejercicio anterior).
Escribe el código asociado para realizar todos los cálculos.**

Para devolver los valores que más se han repetido, definimos una función que reciba el dataframe, el tipo de consulta que se quiere hacer (`Number` o `Star`) y el número `n` de los más repetidos.

Para el cálculo, se realiza un filtro sobre el dataframe sobre el tipo de consulta (`Number` o `Star`), se selecciona la columna de interés `NUMBER`, se le aplica `value_counts` para saber cuantas veces se repite cada valor, y se devuelven los índices de las `n` primeras ya que están ordenadas de mayor a menor. Se pasa a lista para una mejor lectura.

In [26]:
def most_n_repeated_values(df, type_, n):
    return df[df['TYPE']==type_]['NUMBER'].value_counts()[:n].keys().tolist()

Sacamos los 5 números más repetidos y las 2 estrellas ejecutando la función definida anteriormente:

In [27]:
print('Los 5 números más repetidos son {0}. \nLas 2 estrellas más repetidas son {1}.'.format(
    most_n_repeated_values(datos, 'Number', 5), most_n_repeated_values(datos, 'Star', 2)
))

Los 5 números más repetidos son [23, 44, 50, 19, 4]. 
Las 2 estrellas más repetidas son [2, 8].


Para realizar la misma consulta filtrando por año o mes, mejoramos la función anterior añadiendo un nuevo parámetro `by`. En esta función ya hay que hacer más cosas:
- En primer lugar, creamos el dict que vamos a devolver `result`.
- Hacemos el filtro por tipo para descartar las filas que no nos interesan, lo agrupamos y contamos los valores, ahí tendremos un `pandas Series` agrupado por el grupo de interés (año o mes) y el número.
- Seguidamente, guardamos en una lista los diferentes valores por los que vamos a agrupar
- A continuación, por cada grupo, buscamos los n valores más repetidos y devolvemos las claves en formato `dict`

In [28]:
def most_n_repeated_values_by(df, type_, n, by):
    result = {}
    df_grouped = df[df['TYPE']==type_].groupby(by)['NUMBER'].value_counts()
    list_groups = df[by].unique().tolist()
    for group in list_groups:
        result[group] = df_grouped[group][:n].keys().tolist()
    return result

Comprobamos los resultados:

5 números que más han salido por año:

In [29]:
most_n_repeated_values_by(datos, 'Number', 5, 'YEAR')

{2004: [1, 15, 4, 10, 37],
 2005: [11, 50, 3, 26, 47],
 2006: [50, 9, 12, 1, 3],
 2007: [41, 22, 25, 7, 17],
 2008: [50, 19, 45, 7, 37],
 2009: [20, 30, 4, 5, 14],
 2010: [38, 46, 4, 36, 9],
 2011: [12, 23, 50, 14, 28],
 2012: [10, 44, 23, 25, 27],
 2013: [13, 28, 42, 11, 43],
 2014: [13, 38, 3, 4, 25],
 2015: [30, 29, 39, 17, 14],
 2016: [37, 10, 32, 27, 28],
 2017: [17, 30, 20, 4, 10],
 2018: [15, 17, 23, 48, 44],
 2019: [1, 39, 42, 3, 32],
 2020: [11, 5, 15, 46, 27]}

5 números que más han salido por mes:

In [30]:
most_n_repeated_values_by(datos, 'Number', 5, 'MONTH')

{1: [10, 27, 44, 19, 30],
 2: [3, 30, 50, 14, 28],
 3: [23, 44, 4, 36, 17],
 4: [24, 25, 26, 50, 44],
 5: [26, 20, 5, 7, 25],
 6: [39, 7, 11, 17, 34],
 7: [23, 49, 4, 11, 15],
 8: [42, 31, 37, 50, 5],
 9: [6, 14, 38, 35, 44],
 10: [23, 20, 21, 12, 32],
 11: [14, 23, 36, 10, 17],
 12: [31, 43, 44, 8, 1]}

2 estrellas que más han salido por año:

In [31]:
most_n_repeated_values_by(datos, 'Star', 2, 'YEAR')

{2004: [5, 6],
 2005: [1, 3],
 2006: [3, 1],
 2007: [6, 8],
 2008: [4, 5],
 2009: [3, 5],
 2010: [7, 3],
 2011: [2, 5],
 2012: [2, 8],
 2013: [2, 5],
 2014: [1, 10],
 2015: [8, 10],
 2016: [2, 10],
 2017: [3, 9],
 2018: [12, 4],
 2019: [2, 6],
 2020: [6, 2]}

2 estrellas que más han salido por mes:

In [32]:
most_n_repeated_values_by(datos, 'Star', 2, 'MONTH')

{1: [8, 4],
 2: [9, 2],
 3: [6, 9],
 4: [5, 2],
 5: [6, 9],
 6: [1, 2],
 7: [3, 8],
 8: [5, 8],
 9: [1, 9],
 10: [8, 3],
 11: [2, 3],
 12: [2, 3]}

### Ejercicio 4

**Para realizar todos estos procesamientos de cálculo de estadísticas y elementos más repetidos, ¿has exportado la estructura inicial a alguna otra estructura que te facilite el procesamiento? Justifica la respuesta**

Sí, tanto para optimizar el espacio que ocupa el DataFrame como para facilitar el procesamiento, el formato de fecha contenido en el dataframe original: `dd/mm/yyyy` ha sido reemplazado por números enteros. En concreto, se ha dividido la fecha en tres columnas que son `DAY`, `MONTH`, y `YEAR` de forma que a la hora de agrupar es mucho más sencillo hacerlo tanto por mes como por año. De esta forma no hay que estar jugando con fechas, que ralentizaría mucho el procesamiento y las operaciones que queremos realizar. 

Asímismo, se han unificado en la misma columna todos los numeros (tanto los numeros como las estrellas), de forma que toda la información está disponible para poder ser filtrada o agrupada por las demás columnas y pudiendo hacer las operaciones sobre ella.

### Ejercicio 5

**Implementa una función que dada una apuesta, genere una tabla de posibles premios que dicha apuesta hubiera obtenido en los sorteos de los que se dispone de información. Se deberá indicar tanto la categoría de premio como la fecha del sorteo y se omitirán aquellos sorteos en los que la apuesta no hubiera resultado premiada. La función debe de comprobar que la apuesta sea válida antes de buscar los premios.**

Antes de nada, creamos la estructura de los premios y categorías. Como valores tenemos los numeros y estrellas acertados y como índice la categoría:

In [33]:
prizes = pd.DataFrame(np.array(
    [[1,5,2],[2,5,1],[3,5,0],[4,4,2],[5,4,1],[6,3,2],[7,4,0],[8,2,2],[9,3,1],[10,3,0],[11,1,2],[12,2,1],[13,2,0]]
), columns=['Category','Numbers', 'Stars'], )

prizes

,Category,Numbers,Stars
0,1,5,2
1,2,5,1
2,3,5,0
3,4,4,2
4,5,4,1
5,6,3,2
6,7,4,0
7,8,2,2
8,9,3,1
9,10,3,0


En primer lugar realizamos todo tipo de comprobaciones en las siguientes funciones:
- `check_valid_number_format`: Devuelve `True` si la lista de números son de formato entero, si no hay números repetidos y todos ellos están en el rango 1-50.
- `check_valid_star_format`: Devuelve `True` si la lista de estrellas son de formato entero, si no hay números repetidos y todos ellos están en el rango 1-12.
- `check_valid_bet_format`: Devuelve `True` si las funciones `check_valid_number_format` y `check_valid_star_format`  devuelven `True` y si la lista de numeros y la lista de estrellas tienen la longitud que deben tener, 50 y 12 respectivamente.

In [34]:
def check_valid_number_format(list_numbers):
    try:
        for number in list_numbers:
            number = int(number)
        for number in list_numbers:
            if(list_numbers.count(number) > 1):
                print('No puede haber números repetidos')
                return False
            if(0>=number or number>50):
                print('El número debe estar en el rango 1-50')
                return False
        return True
    except ValueError:
        print('El número debe ser un entero')
        return False
        
def check_valid_star_format(list_stars):
    try:
        for star in list_stars:
            star = int(star)
        for star in list_stars:
            if(list_stars.count(star) > 1):
                print('No puede haber estrellas repetidas')
                return False
            if(0>=star or star>12):
                print('La estrella debe estar en el rango 1-12')
                return False
        return True
    except ValueError:
        print('La estrella debe ser un número entero')
        return False

                
def check_valid_bet_format(list_numbers, list_stars):
    if(len(list_numbers)!=5):
        print('La lista de números debe tener una longitud de 5 números para que la apuesta sea válida')
        return False
    elif(len(list_stars)!=2):
        print('La lista de estrellas debe tener una longitud de 2 números para que la apuesta sea válida')
        return False
    return check_valid_number_format(list_numbers) and check_valid_star_format(list_stars)  

Seguidamente, se va a implementar la función `posible_win_results` que va a devolver los resultados en los que la combinación de entrada `numbers` y `stars` podría haber sido premiada. Indicando la categoría de premio (de acuerdo con la tabla del enunciado) y la fecha.



Antes de implementar la función, vamos a ver cual es la mejor forma de implementarla, sabiendo que vamos a recibir dos listas.

La mejor forma para trabajar con números no repetidos y saber cuantas coincidencias ha habido, es conocer cual es la intersección entre la apuesta ganadora y la apuesta realizada y calcular su longitud. Por lo tanto, en primer lugar nos interesa agrupar nuestro DataFrame por año, mes, dia y tipo de dato, y como valor obtendremos el `set` de los números contenidos en la columna `NUMBER`, ya que como hemos comentado es muy facil trabajar con sets para este tipo de operaciones:

In [35]:
ex5_sample = datos.groupby(['YEAR', 'MONTH', 'DAY', 'TYPE']).NUMBER.apply(lambda x: set(x))

Echamos un ojo a la nueva estructura:

In [36]:
ex5_sample.head()

YEAR  MONTH  DAY  TYPE  
2004  2      13   Number    {16, 41, 32, 36, 29}
                  Star                    {9, 7}
             20   Number     {39, 47, 50, 13, 7}
                  Star                    {2, 5}
             27   Number    {18, 19, 37, 14, 31}
Name: NUMBER, dtype: object

Supongamos que la apuesta realizada es {16, 20, 42, 44, 47} y {8, 1}, obtendremos los números acertados sacanco la intersección entre conjuntos (`set`):

In [37]:
ex5_sample_nums = ex5_sample.loc[:, :, :, 'Number'].apply(lambda x: x.intersection({16, 20, 42, 44, 47}))
ex5_sample_nums.head()

YEAR  MONTH  DAY
2004  2      13         {16}
             20         {47}
             27           {}
      3      5            {}
             12     {44, 47}
Name: NUMBER, dtype: object

Y su longitud:

In [38]:
ex5_sample_nums = ex5_sample.loc[:, :, :, 'Number'].apply(lambda x: len(x.intersection({16, 20, 42, 44, 47})))
ex5_sample_nums.name = 'Numbers'
ex5_sample_nums.head()

YEAR  MONTH  DAY
2004  2      13     1
             20     1
             27     0
      3      5      0
             12     2
Name: Numbers, dtype: int64

Lo mismo con las estrellas habrá que hacer con las estrellas.

In [39]:
ex5_sample_stars = ex5_sample.loc[:, :, :, 'Star'].apply(lambda x: len(x.intersection({9, 2})))
ex5_sample_stars.name = 'Stars'
ex5_sample_stars.head()

YEAR  MONTH  DAY
2004  2      13     1
             20     1
             27     0
      3      5      0
             12     0
Name: Stars, dtype: int64

Tendremos entonces dos `pandas.Series` con las mismas claves y como valor el número de coincidencias de los números y de las estrellas.

Vamos a fusionarlos en un dataframe:

In [40]:
ex5_matched = pd.concat([ex5_sample_nums, ex5_sample_stars], axis=1)
ex5_matched.head()

Numbers  Stars
YEAR MONTH DAY                
2004 2     13         1      1
           20         1      1
           27         0      0
     3     5          0      0
           12         2      0

Y comparamos con los premios para obtener un nuevo dataframe con las fechas y la categoria del premio si existe. Para ello hacemos un merge de ambos dataframes creando antes una columna con el valor del indice para no perder la fecha:

In [41]:
ex5_matched.reset_index(inplace=True)
ex5_matched.merge(prizes).tail()

,YEAR,MONTH,DAY,Numbers,Stars,Category
96,2009,12,11,2,2,8
97,2014,1,28,2,2,8
98,2018,3,6,2,2,8
99,2010,10,29,3,1,9
100,2015,5,15,3,1,9


Y ahí lo tendríamos.

Para la implementación de la función. Esta recibe como parametros de entrada el dataframe `df`, los numeros jugados `numbers`, las estrellas jugadas `stars` y la categorización de premios `prizes_`:
- En primer lugar, comprobamos que el formato de la apuesta es correcto con la funcion `check_valid_bet_format`. (1)
- Seguidamente obtenemos los conjuntos agrupados de las coincidencias que han habido de numeros y de estrellas y les damos nombre. (2)
- Despues, lo unificamos en un unico dataset (3)
- Por último, hacemos un reindex manteniendo la fecha como columnas y mergeamos con los premios para obtener las filas premiadas (4)


In [42]:
def posible_win_results(df, numbers, stars, prizes_):
    # (1)
    if(not check_valid_bet_format(numbers,stars)):
        return 0
    set_numbers = set(numbers)
    set_stars = set(stars)
    # (2)
    grouped_df = datos.groupby(['YEAR', 'MONTH', 'DAY', 'TYPE']).NUMBER.apply(lambda x: set(x))
    matched_numbers = grouped_df.loc[:, :, :, 'Number'].apply(
        lambda x: len(set(x).intersection(set_numbers))
    )
    matched_numbers.name = 'Numbers'
    matched_stars = grouped_df.loc[:, :, :, 'Star'].apply(
        lambda x: len(set(x).intersection(set_stars))
    )
    matched_stars.name = 'Stars'
    # (3)
    df_matched = pd.concat([matched_numbers, matched_stars], axis=1)
    # (4)
    df_matched.reset_index(inplace=True)
    final_result = df_matched.merge(prizes_)
    return final_result

Para probarlo:

In [43]:
posible_win_results(datos, [15,20,26,31,44], [2, 13], prizes)

La estrella debe estar en el rango 1-12


0

In [44]:
posible_win_results(datos, [15,20,26,31,44], [2, 9], prizes)

,YEAR,MONTH,DAY,Numbers,Stars,Category
0,2004,3,12,2,0,13
1,2004,7,16,2,0,13
2,2004,8,6,2,0,13
3,2004,9,24,2,0,13
4,2005,4,1,2,0,13
5,2005,5,6,2,0,13
6,2005,12,23,2,0,13
7,2006,3,31,2,0,13
8,2006,8,4,2,0,13
9,2007,7,13,2,0,13


### Ejercicio 6

**Crea una apuesta formada por los 5 números y las 2 estrellas más repetidas obtenidas en el ejercicio 3. ¿Qué premios habría obtenido dicha apuesta en todos los sorteos?**

En primer lugar, creamos un dict con los datos del ejercicio 3 (los 5 números y las 2 estrellas más repetidas):

In [45]:
most_repeated_bet = {
    'numbers' : most_n_repeated_values(datos, 'Number', 5),
    'stars' : most_n_repeated_values(datos, 'Star', 2)
}

In [46]:
most_repeated_bet

{'numbers': [23, 44, 50, 19, 4], 'stars': [2, 8]}

Y a continuación, lanzamos la función que comprueba los posibles premios:

In [47]:
results = posible_win_results(datos, most_repeated_bet['numbers'], most_repeated_bet['stars'], prizes)

In [48]:
results.head()

,YEAR,MONTH,DAY,Numbers,Stars,Category
0,2004,3,26,2,1,12
1,2004,7,9,2,1,12
2,2005,1,7,2,1,12
3,2006,1,20,2,1,12
4,2006,2,17,2,1,12


Y creamos una función que nos imprima el resultado:

In [49]:
def print_results(res):
    print('La apuesta ha sido premiada un total de {0} veces'.format(len(res)))
    for x in range(13):
        res_cat = res[res['Category']==x+1]
        print('Han habido {0} premios de categoría {1}'.format(len(res_cat), x+1))
        if(len(res_cat)>0):
            print(res_cat[['DAY', 'MONTH', 'YEAR']])

In [50]:
print_results(results)

La apuesta ha sido premiada un total de 145 veces
Han habido 0 premios de categoría 1
Han habido 0 premios de categoría 2
Han habido 0 premios de categoría 3
Han habido 0 premios de categoría 4
Han habido 0 premios de categoría 5
Han habido 1 premios de categoría 6
    DAY  MONTH  YEAR
49    9      4  2004
Han habido 0 premios de categoría 7
Han habido 2 premios de categoría 8
     DAY  MONTH  YEAR
143   17      8  2012
144   28     10  2016
Han habido 6 premios de categoría 9
     DAY  MONTH  YEAR
121   27     10  2006
122    3     10  2008
123   22      7  2011
124    1     10  2013
125   11      3  2014
126    7      9  2018
Han habido 6 premios de categoría 10
     DAY  MONTH  YEAR
127   12      9  2008
128   16      3  2012
129   16      7  2013
130   26     11  2013
131    3     10  2014
132   31      1  2017
Han habido 10 premios de categoría 11
     DAY  MONTH  YEAR
133   30      7  2010
134   24     12  2010
135   25      3  2014
136   25      9  2015
137   13     12  2016
138

### Ejercicio 7

**Queremos generar aleatoriamente 100 apuestas y guardarlas en un fichero. ¿Qué estructura de datos, de entre las estudiadas, usarías y cómo lo harías? Justifica tus decisiones (entre ellas el formato de salida usado en el fichero).** 

**Escribe el código necesario para crear la estructura y generar las distintas combinaciones. También escribe el código para guardar el contenido en un fichero.**

Para guardar en un fichero:
- `bet_id`: Id de la apuesta. No puede ser negativo. El id se repetirá para todos los números que pertenezcan a la misma apuesta. Rango 1-100 para nuestro caso.
- `TYPE`: Tipo de numero (`Number` o `Star`)
- `NUMBER`: Numero que ha salido. Rango 1-50 si `TYPE='Number'` o rango 1-12 si `TYPE='Star'`. `uint8`.

Seguidamente, lo exportaría a **csv** (comma separated value).

Esta estructura nos permite poder realizar con facilidad operaciones que consisten en agrupaciones. Sacar los n valores más repetidos por ejemplo.

No obstante, para saber cuales han sido las premiadas es más facil trabajar con la estructura de conjuntos **(`set`)** ya que nos permite realizar de forma muy sencilla operaciones de comparacion con las soluciones sacando la intersección entre la apuesta y la apuesta ganadora. Además, cumple exactamente con las características que necesitamos. Su contenido no puede repetirse.

Para ello, vamos a implementar :
- Una función `optimize_bets_dataframe`que optimice nuestro dataframe como se ha comentado:

In [51]:
def optimize_bets_dataframe(df):
    df['TYPE'] = df['TYPE'].astype('category')
    df['NUMBER'] = df['NUMBER'].astype('uint8')
    return df

- Una función `generate_unique_combination` que genere aleatoriamente una combinación de `n` números enteros no repetidos en el rango `min_` inclusivo y `max_` exclusivo. Devuelve un `set` con la combinación. *Nota: Para evitar que los números se repitan se usará la función random.sample(population, k, * counts=None)*

In [52]:
def generate_unique_combination(n, min_, max_):
    return set(random.sample(range(min_,max_), n))

- Una función `generate_random_bets` que genere aleatoriamente 100 apuestas aleatorias. Esta función va a recibir como parámetro de entrada el número de apuestas aleatorias `n` (por defecto 100) que queremos generar y como salida va a devolver un dataset con la estructura final que queremos tener.

In [53]:
def generate_random_bets(n=100):
    list_ = []
    for i in range(n):
        combination_numbers = generate_unique_combination(5, 1, 50)
        combination_stars = generate_unique_combination(2, 1, 12)
        for idx, val in enumerate(combination_numbers):
            list_.append([i, 'Number', val])
        for idx, val in enumerate(combination_stars):
            list_.append([i, 'Star', val])
    df = pd.DataFrame(list_, columns=['bet_id', 'TYPE', 'NUMBER'])
    df = optimize_bets_dataframe(df)
    return df

- Una función `create_and_export_bets` que cree el dataframe y lo llene con las `n` apuestas generadas aleatoriamente. Seguidamente, lo exportará a csv con el nombre `name`. Si no hemos pasado dicho parámetro, no lo guardará:

In [54]:
def create_and_export_bets(n=100, name=None):
    df = generate_random_bets(n)
    if(name is not None):
        df.to_csv(name+'.csv', index=False)
    return df

In [55]:
# Se hace uso del head para que la respuesta no ocupe demasiado espacio en el notebook
create_and_export_bets(100, 'random_100_bets').head()

,bet_id,TYPE,NUMBER
0,0,Number,33
1,0,Number,26
2,0,Number,37
3,0,Number,13
4,0,Number,46


> Compruébese que el fichero ha sido generado y exportado

- Una función `import_bets` que importe un dataframe exportado a csv con el nombre `name` con el formato descrito anteriormente y lo devuelva como DataFrame. Se hace uso del parámetro `converters` de `read_csv` para que carge como `set` las columnas y no como `str`.

In [56]:
def import_bets(name=None):
    if(name is not None):
        df = pd.read_csv(name+'.csv')
        return optimize_bets_dataframe(df)
    else:
        return None

Cargamos las apuestas que hemos exportado anteriormente:

In [57]:
bets = import_bets('random_100_bets')
# Se hace uso del head para que la respuesta no ocupe demasiado espacio en el notebook
bets.head()

,bet_id,TYPE,NUMBER
0,0,Number,33
1,0,Number,26
2,0,Number,37
3,0,Number,13
4,0,Number,46


In [58]:
bets.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 3 columns):
bet_id    700 non-null int64
TYPE      700 non-null category
NUMBER    700 non-null uint8
dtypes: category(1), int64(1), uint8(1)
memory usage: 7.1 KB


### Ejercicio 8
**A partir de los datos generados en el ejercicio 7, obtén los números y las estrellas más repetidos.**

Para sacar los números y estrellas más repetidos solamente tenemos que llamar a las funciones creadas en ejercicios anteriores de la siguiente forma:
- `most_n_repeated_values(bets, 'Number', 5)`
- `most_n_repeated_values(bets, 'Star', 2)`

In [59]:
print('Los 5 números más repetidos son {0}. \nLas 2 estrellas más repetidas son {1}.'.format(
    most_n_repeated_values(bets, 'Number', 5), most_n_repeated_values(bets, 'Star', 2)
))

Los 5 números más repetidos son [37, 18, 46, 43, 2]. 
Las 2 estrellas más repetidas son [6, 8].


### Ejercicio 9
**Supongamos que jugamos con las 100 apuestas generadas aleatoriamente. ¿Qué premios, y en qué fechas, habríamos obtenido con respecto al histórico que teníamos del principio? Piensa en al menos dos formas distintas de mostrar la información, discutiendo las ventajas e inconvenientes de cada una de ellas.**

Como en ejercicios anteriores, para obtener los posibles resultados habría que ejecutar 100 veces la siguiente función con los datos de cada una de las 100 apuestas:

```
posible_win_results(df, numbers, stars, prizes)
```

Para recoger la información (por ejemplo, las estrellas de la 5a apuesta):

In [60]:
# En primer lugar filtramos por la 5a apuesta
fifth_bet = bets[bets['bet_id']==5]
fifth_bet

,bet_id,TYPE,NUMBER
35,5,Number,19
36,5,Number,20
37,5,Number,6
38,5,Number,30
39,5,Number,14
40,5,Star,11
41,5,Star,7


In [61]:
# Seguidamente por la categoria estrella
fifth_stars = fifth_bet[fifth_bet['TYPE']=='Star']
fifth_stars

,bet_id,TYPE,NUMBER
40,5,Star,11
41,5,Star,7


In [62]:
# Finalmente obtenemos la lista
fifth_stars['NUMBER'].tolist()

[11, 7]

Por lo tanto, vamos a ejecutar la funcion anterior 'n=100' veces para obtener todas las apuestas ganadoras. 

La primera forma de presentar la información es tal y como lo hemos hecho en el ejercicio 5:

In [63]:
def print_possible_results(bets_):
    # Sacamos todos los ids de las apuestas
    ids = bets_['bet_id'].unique()
    # Y las recorremos
    for i in ids:
        ith_bet = bets_[bets_['bet_id']==i]
        ith_numbers = ith_bet[ith_bet['TYPE']=='Number']['NUMBER'].tolist()
        ith_stars = ith_bet[ith_bet['TYPE']=='Star']['NUMBER'].tolist()
        print('La apuesta número {0} cuyos números son {1} y sus estrellas {2}:'.format(
            i, ith_numbers, ith_stars
        ))
        print_results(posible_win_results(datos, ith_numbers, ith_stars, prizes))

In [64]:
print_possible_results(bets)

La apuesta número 0 cuyos números son [33, 26, 37, 13, 46] y sus estrellas [11, 7]:
La apuesta ha sido premiada un total de 101 veces
Han habido 0 premios de categoría 1
Han habido 0 premios de categoría 2
Han habido 0 premios de categoría 3
Han habido 0 premios de categoría 4
Han habido 0 premios de categoría 5
Han habido 1 premios de categoría 6
     DAY  MONTH  YEAR
100   12      9  2014
Han habido 0 premios de categoría 7
Han habido 0 premios de categoría 8
Han habido 1 premios de categoría 9
    DAY  MONTH  YEAR
89   12      3  2010
Han habido 4 premios de categoría 10
    DAY  MONTH  YEAR
90   23      7  2010
91   21     10  2011
92   16      8  2016
93   28      3  2017
Han habido 6 premios de categoría 11
    DAY  MONTH  YEAR
94   31      8  2012
95   15      4  2014
96   11     11  2014
97    3      2  2015
98    2     11  2018
99    2      6  2020
Han habido 22 premios de categoría 12
    DAY  MONTH  YEAR
67   12      9  2008
68   10     12  2010
69   11      3  2011
70   18 

Con esta forma obtenemos toda la información que nos interesa. No obstante, no es muy clara dada la gran cantidad de datos que queremos visualizar. Deberíamos tratar de reducir la dimensionalidad y mostrar el resultado con un DataFrame. Este será de una dimensión muy grande si queremos mostrar todas las apuestas, así que lo mejor será crear una función que reciba el id de apuesta (de 0 a 100) y muestre en un dataframe los resultados que habría obtenido:

In [65]:
def print_possible_results_2(bets_, id_bet):
    ith_bet = bets_[bets_['bet_id']==id_bet]
    ith_numbers = ith_bet[ith_bet['TYPE']=='Number']['NUMBER'].tolist()
    ith_stars = ith_bet[ith_bet['TYPE']=='Star']['NUMBER'].tolist()
    print('La apuesta número {0} cuyos números son {1} y sus estrellas {2}:'.format(
        id_bet, ith_numbers, ith_stars
    ))
    return posible_win_results(datos, ith_numbers, ith_stars, prizes)

In [66]:
print_possible_results_2(bets, 5)

La apuesta número 5 cuyos números son [19, 20, 6, 30, 14] y sus estrellas [11, 7]:


,YEAR,MONTH,DAY,Numbers,Stars,Category
0,2004,2,27,2,0,13
1,2005,10,14,2,0,13
2,2005,10,21,2,0,13
3,2006,1,6,2,0,13
4,2006,11,10,2,0,13
5,2006,12,29,2,0,13
6,2007,1,12,2,0,13
7,2007,2,2,2,0,13
8,2007,2,9,2,0,13
9,2007,2,16,2,0,13
